# Show the data

In [1]:
import os 
os.chdir('iron_handmaidens')
import pandas as pd
from data.src.emg import EMGData
import plotly.graph_objects as go
import plotly.express as px

In [2]:
left = EMGData.read_csv('/Users/aldenmoreton/Documents/iron_handmaidens/test_data/participant1/A/Shimmer_4680.csv')
right = EMGData.read_csv('/Users/aldenmoreton/Documents/iron_handmaidens/test_data/participant1/A/Shimmer_5470.csv')
center = left + right

FileNotFoundError: [Errno 2] No such file or directory: '/Users/aldenmoreton/Documents/iron_handmaidens/test_data/participant1/A/Shimmer_4680.csv'

In [ ]:
left.df.rolling(30).mean()

NameError: name 'left' is not defined

In [ ]:
left.preprocess().figure().show()

In [ ]:
left.quartiles()

,CH1_4680,CH2_4680
0.9,6.383125,8.142892
0.5,2.389029,1.781044
0.1,0.204104,-2.327004


In [ ]:
toggles = left.preprocess().df
toggles['Toggle'] = toggles[left['Event']].diff()
toggles = toggles[abs(toggles['Toggle']) == 3]
toggles = toggles.reset_index()
toggles
li = [(toggles.iloc[i], toggles.iloc[i+1]) for i in range(0, len(toggles)-1, 2)]
for start, stop in li:
	print(start['index'], stop['index'])

39268.0 45657.0
86663.0 94785.0
469831.0 1415977.0
1449111.0 1456384.0
1512873.0 1520821.0


In [ ]:
testEMGData = left.preprocess()
fig = go.Figure()
plots = []
for line in testEMGData.find_columns('CH'):
	newFig = go.Scatter(x=testEMGData.df[testEMGData['Elapse']].iloc[::100], y=testEMGData.df[line].iloc[::100], name=line, visible='legendonly')
	plots.append(newFig)
[fig.add_trace(plot) for plot in plots]
fig.for_each_trace(lambda trace: trace.update(visible=True) if trace.name == 'CH1_4680' else ())

for i in range(0, len(toggles)-1, 2):
	fig.add_vrect(toggles[testEMGData['Elapse']].iloc[i], toggles[testEMGData['Elapse']].iloc[i+1], fillcolor='green', opacity=0.15)

fig.show()

In [ ]:
def find_events(self):
	new = pd.DataFrame()

	new['Toggle'] = self.df[self['Event']].diff()
	new = self.df.loc[abs(new['Toggle']) == 3]
	
	return [(new.iloc[i], new.iloc[i+1]) for i in range(0, len(new)-1, 2)]

In [ ]:
for start, stop in find_events(right.preprocess()):
	print(start['Elapse (s)'], stop['Elapse (s)'])

38.39781469726562 44.56861328125
84.76620336914063 92.63521655273438
458.7376774902344 1382.7959333496094
1415.131346435547 1422.2503715820312
1477.4067036132813 1485.283529296875
